# Task1. Baseline Methods

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-10-20  

## 1. Import packages

In [3]:
import numpy as np
import pandas as pd
import time
import datetime
import sys
import os
from tqdm import tqdm
from functools import reduce

sys.path.append("../tools/")
import funclib

sys.path.append("../")
import benchmark_train as btrain
import benchmark_test as btest
import config as cfg
import benchmark_evaluation as eva

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from pandarallel import pandarallel #  import pandaralle
pandarallel.initialize() # init

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Load Data

In [2]:
test = pd.read_feather(cfg.DATADIR+'task1/test.feather')
print('test size: {0}'.format(len(test)))

test size: 7101


## 3.Make fasta

In [9]:
funclib.table2fasta(table=test, file_out='../data/test.fasta')

Write finished


## 4. ECPred

<span style="color:red" >Please be patient, this method takes a long time to predict</span>

In [3]:
!sudo apt-get update -y
!sudo apt-get install default-jre -y
!sudo apt-get install default-jdk -y

Hit:1 http://cn.archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://cn.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://cn.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 http://cn.archive.ubuntu.com/ubuntu focal-security InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-72).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jdk is already the newest version (2:1.11-72).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
!java -jar ../baselines/ECPred/ECPred.jar weighted ../data/test.fasta ../baselines/ECPred/ ../results/ecpred.txt

Main classes of input proteins are being predicted ...


## 5. DeepEC

In [ ]:
! conda env create -f ../baselines/deepec/environment.yml
! conda activate deepec

In [ ]:
! python ../baselines/deepec/deepec.py -i ../data/test.fasta -o ../results/deepec/

## 6. CatFam

In [1]:
!sudo apt-get install ncbi-blast+-legacy -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmbedcrypto3 libmbedtls12 libmbedx509-0 ncbi-blast+ ncbi-data
The following NEW packages will be installed:
  libmbedcrypto3 libmbedtls12 libmbedx509-0 ncbi-blast+ ncbi-blast+-legacy
  ncbi-data
0 upgraded, 6 newly installed, 0 to remove and 44 not upgraded.
Need to get 14.9 MB of archives.
After this operation, 75.0 MB of additional disk space will be used.
Get:1 http://cn.archive.ubuntu.com/ubuntu focal/universe amd64 libmbedcrypto3 amd64 2.16.4-1ubuntu2 [150 kB]
Get:2 http://cn.archive.ubuntu.com/ubuntu focal/universe amd64 libmbedx509-0 amd64 2.16.4-1ubuntu2 [42.3 kB]
Get:3 http://cn.archive.ubuntu.com/ubuntu focal/universe amd64 libmbedtls12 amd64 2.16.4-1ubuntu2 [71.8 kB]
Get:4 http://cn.archive.ubuntu.com/ubuntu focal/universe amd64 ncbi-data all 6.1.20170106+dfsg1-8 [3,518 kB]
Get:5 http://cn.archive.ubuntu.com/ubuntu focal/uni

In [21]:
! ../baselines/catfam/source/catsearch.pl -d ../baselines/catfam/CatFamDB/CatFam_v2.0/CatFam4D99R -i ../data/test.fasta -o ../results/catfam.txt

In [ ]:
! java -Xmx50G -jar ../baselines/priam/PRIAM_search.jar -p /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_JAN18 -i /home/shizhenkun/codebase/DMLF/data/gu_bang.fasta -o /home/shizhenkun/codebase/DMLF/results/case_gubang/priam/ --bp /home/shizhenkun/downloads/blast-2.2.13/bin --np 78

## 7. PRIAM

In [26]:
! java -Xmx50G -jar ../baselines/priam/PRIAM_search.jar -p /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_JAN18 -i /home/shizhenkun/codebase/DMLF/data/test.fasta -o /home/shizhenkun/codebase/DMLF/results/priam/ --bp /home/shizhenkun/downloads/blast-2.2.13/bin --np 78



PRIAM did not find the profiles library (Normal if it is the first time you use PRIAM with this release).
So it would now create it. Please do not interupt it during this process else it would results into a corrupted database and any further analysis would be faulty.
If for any reason you really need to interupt PRIAM before it ends, or if your computer crash during this process, please delete ../baselines/priam/PRIAM_JAN18/PROFILES/LIBRARY, and all files it contains, to force PRIAM to recreate the library next time it would be launched

Executing makeprofiledb -in /home/shizhenkun/codebase/DMLF/tools/../baselines/priam/PRIAM_JAN18/PROFILES/LIBRARY/profiles.list -index T -out PROFILE_EZ -title PRIAM_profiles_database 

Profiles database sucessfully created.
If you want to interupt PRIAM, it can now be done with no risk

### PRIAM Profiles search ###
7101 sequences found in your query file
Query file splitted into 2 pieces
Executing 'rpsblast -num_threads 78 -outfmt 5 -evalue 1.0 -qu